# Single Sample Method Benchmarking

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>).
- **Affiliation:** Helmholtz Munich, Institute of Computational Biology (ICB), Talavera-López Lab
- **Date of Creation:** 06.01.2023
- **Date of Last Modification:** 16.08.2023

## 1. Setup

### 1.1 Import Libraries

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys
sys.path.append("../../../nichecompass-reproducibility/utils")

In [1]:
import os
import shutil
import warnings
from datetime import datetime

import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import pickle
import plottable
import scanpy as sc
import scib_metrics
import seaborn as sns
from matplotlib import gridspec
from matplotlib.colors import LinearSegmentedColormap
from plottable import ColumnDefinition, Table
from plottable.cmap import normed_cmap
from plottable.formatters import tickcross
from plottable.plots import bar

from nichecompass.utils import (add_gps_from_gp_dict_to_adata,
                                extract_gp_dict_from_mebocost_es_interactions,
                                extract_gp_dict_from_nichenet_lrt_interactions,
                                extract_gp_dict_from_omnipath_lr_interactions,
                                filter_and_combine_gp_dict_gps)

#from benchmarking_utils import (compute_combined_benchmarking_metrics_for_all_models,
#                                compute_combined_benchmarking_metrics)

### 1.2 Define Parameters

In [27]:
spatial_key = "spatial"

### 1.3 Run Notebook Setup

In [28]:
sc.set_figure_params(figsize=(6, 6))

In [29]:
# Ignore future warnings and user warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)

In [30]:
# Get time of notebook execution for timestamping saved artifacts
now = datetime.now()
current_timestamp = now.strftime("%d%m%Y_%H%M%S")

In [31]:
# Set mlflow tracking server (run it on the defined port)
mlflow.set_tracking_uri("http://localhost:8889")

### 1.4 Configure Paths and Directories

In [4]:
data_folder_path = "../../datasets/srt_data/gold"
artifact_folder_path = f"../../artifacts"
gp_data_folder_path = "../../datasets/gp_data" # gene program data
nichenet_ligand_target_mx_file_path = gp_data_folder_path + "/nichenet_ligand_target_matrix.csv"
omnipath_lr_interactions_file_path = gp_data_folder_path + "/omnipath_lr_interactions.csv"
benchmarking_folder_path = f"{artifact_folder_path}/single_sample_method_benchmarking"

## 2. Method Benchmarking

- Run all model notebooks in the ```notebooks/single_sample_method_benchmarking``` directory before continuing.

### 2.1 Retrieve NicheCompass Runs

#### 2.1.1 seqFISH Mouse Organogenesis

In [14]:
# Store NicheCompass results in task folder
task = "single_sample_method_benchmarking"
conv_layer_encoder = "gcnconv"
datasets = [
    "seqfish_mouse_organogenesis_embryo2",
    "seqfish_mouse_organogenesis_subsample_50pct_embryo2",
    "seqfish_mouse_organogenesis_subsample_25pct_embryo2",
    "seqfish_mouse_organogenesis_subsample_10pct_embryo2",
    "seqfish_mouse_organogenesis_subsample_5pct_embryo2",
    "seqfish_mouse_organogenesis_subsample_1pct_embryo2",
]
timestamps = [
    "16082023_152524_1",
    "16082023_152524_1",
    "16082023_152524_1",
    "16082023_152524_1",
    "16082023_152524_1",
    "16082023_152524_1",
]

for dataset, timestamp in zip(datasets, timestamps):
    source_path = f"{artifact_folder_path}/{dataset}/results/{conv_layer_encoder}_{task}/{timestamp}/{dataset}_{conv_layer_encoder}_{task}.h5ad"
    destination_path = f"{artifact_folder_path}/{task}/{dataset}_nichecompass_{conv_layer_encoder}.h5ad"
    shutil.copy(source_path, destination_path)

FileNotFoundError: [Errno 2] No such file or directory: '../../artifacts/seqfish_mouse_organogenesis_subsample_10pct_embryo2/results/gcnconv_single_sample_method_benchmarking/16082023_152524_1/seqfish_mouse_organogenesis_subsample_10pct_embryo2_gcnconv_single_sample_method_benchmarking.h5ad'

#### 2.1.2 seqFISH Mouse Organogenesis Imputed

In [ ]:
# Store NicheCompass results in task folder
task = "single_sample_method_benchmarking"
conv_layer_encoder = "gcn"
datasets = [
    "seqfish_mouse_organogenesis_imputed_embryo2",
    "seqfish_mouse_organogenesis_imputed__subsample_50pct_embryo2",
    "seqfish_mouse_organogenesis_imputed__subsample_25pct_embryo2",
    "seqfish_mouse_organogenesis_imputed__subsample_10pct_embryo2",
    "seqfish_mouse_organogenesis_imputed__subsample_5pct_embryo2",
    "seqfish_mouse_organogenesis_imputed__subsample_1pct_embryo2",
]
timestamps = [
    "15082023_172034_1",
    "15082023_172034_1",
    "15082023_172034_1",
    "15082023_172034_1",
    "15082023_172034_1",
    "15082023_172420_1",
]

for dataset, timestamp in zip(datasets, timestamps):
    source_path = f"{artifact_folder_path}/{dataset}/results/one-hop-norm_{task}/{timestamp}/{dataset}_one-hop-norm_{task}.h5ad"
    destination_path = f"{artifact_folder_path}/{task}/{dataset}_nichecompass_{conv_layer_encoder}.h5ad"
    shutil.copy(source_path, destination_path)

#### 2.1.3 nanoString CosMx SMI Human Non-Small-Cell Lung Cancer (NSCLC)

In [ ]:
import scanpy as sc
adata = sc.read_h5ad("../artifacts/single_sample_method_benchmarking/nanostring_cosmx_human_nsclc_batch5_one-hop-norm_single_sample_method_benchmarking.h5ad")
adata2 = sc.read_h5ad("../artifacts/single_sample_method_benchmarking/nanostring_cosmx_human_nsclc_batch5_one-hop-norm_single_sample_method_benchmarking_2.h5ad")

adata2.uns["nichecompass_latent_run8_umap"] = adata.uns["nichecompass_latent_run8_umap"]
adata2.uns["nichecompass_model_training_duration_run8"] = adata.uns["nichecompass_model_training_duration_run8"]
adata2.obsm["nichecompass_latent_run8"] = adata.obsm["nichecompass_latent_run8"]
adata2.obsm["nichecompass_latent_run8_X_umap"] = adata.obsm["nichecompass_latent_run8_X_umap"]
adata2.obsp["nichecompass_latent_run8_connectivities"] = adata.obsp["nichecompass_latent_run8_connectivities"]
adata2.obsp["nichecompass_latent_run8_distances"] = adata.obsp["nichecompass_latent_run8_distances"]

adata2.write("../artifacts/single_sample_method_benchmarking/nanostring_cosmx_human_nsclc_batch5_nichecompass_gcn.h5ad")

#### 2.1.4 Vizgen MERFISH Mouse Liver

#### 2.1.5 Slide-seqV2 Mouse Hippocampus

In [ ]:
# Store NicheCompass results in task folder
task = "single_sample_method_benchmarking"
conv_layer_encoder = "gcnconv"
datasets = [
    "slideseqv2_mouse_hippocampus",
    "slideseqv2_mouse_hippocampus_subsample_50pct",
    "slideseqv2_mouse_hippocampus_subsample_25pct",
    "slideseqv2_mouse_hippocampus_subsample_10pct",
    "slideseqv2_mouse_hippocampus_subsample_5pct",
    "slideseqv2_mouse_hippocampus_subsample_1pct",
]
timestamps = [
    "16082023_155038_1",
    "16082023_155611_1",
    "16082023_155603_1",
    "16082023_155604_1",
    "16082023_160141_1",
    "16082023_160422_1",
]

for dataset, timestamp in zip(datasets, timestamps):
    source_path = f"{artifact_folder_path}/{dataset}/results/{conv_layer_encoder}_{task}/{timestamp}/{dataset}_{conv_layer_encoder}_{task}.h5ad"
    destination_path = f"{artifact_folder_path}/{task}/{dataset}_nichecompass_{conv_layer_encoder}.h5ad"
    shutil.copy(source_path, destination_path)

In [11]:
datasets = ["nanostring_cosmx_human_nsclc_subsample_50pct_batch5",
            "nanostring_cosmx_human_nsclc_subsample_1pct_batch5",
            "nanostring_cosmx_human_nsclc_subsample_5pct_batch5",
            #"nanostring_cosmx_human_nsclc_batch5", 39
            "nanostring_cosmx_human_nsclc_subsample_10pct_batch5",
            "nanostring_cosmx_human_nsclc_subsample_25pct_batch5"]
experiment_ids = [36, 37, 38, 40, 41]

In [13]:
# Retrieve metrics and params of ablation runs from mlflow and save to disk
summary_df = pd.DataFrame()
for dataset, experiment_id in zip(datasets, experiment_ids):
    runs_info = mlflow.list_run_infos(experiment_id)
    for run_info in runs_info:
        run = mlflow.get_run(run_info.run_uuid)
        if run.info.status == "RUNNING":
            continue
        run_dict = {"dataset": dataset}
        run_dict["timestamp"] = run.data.params["timestamp"]
        run_dict["run_number"] = run.data.params["run_number"]
        run_dict["val_auroc_score"] = run.data.metrics.get("val_auroc_score", np.nan)
        run_dict["val_target_rna_mse_score"] = run.data.metrics.get("val_target_rna_mse_score", np.nan)
        run_dict["val_source_rna_mse_score"] = run.data.metrics.get("val_source_rna_mse_score", np.nan)
        run_dict["n_neighbors"] = run.data.params["n_neighbors"]
        run_df = pd.DataFrame(run_dict, index=[0])
        summary_df = pd.concat([summary_df, run_df], ignore_index=True)
        summary_df["val_gene_expr_mse_score"] = summary_df.apply(lambda row: (row["val_target_rna_mse_score"] + row["val_source_rna_mse_score"])/2, axis=1)
        summary_df["n_neighbors"] = summary_df["n_neighbors"].astype(str)

for dataset in datasets:
    dataset_df = summary_df[summary_df["dataset"] == dataset].copy()
    dataset_df.to_csv(f"{benchmarking_folder_path}/mlflow_summary_{dataset}_nichecompass_single_sample_method_benchmarking.csv", index=False)

### 2.1 Benchmarking Metrics

#### 2.2.1 Define Functions

In [18]:
adata.uns[
        f"nichecompass_model_training_duration_run7"]

KeyError: 'nichecompass_model_training_duration_run7'

In [40]:
adata = sc.read_h5ad("../../artifacts/single_sample_method_benchmarking/nanostring_cosmx_human_nsclc_subsample_20pct_autotalker_single_sample_method_benchmarking.h5ad")

In [75]:
adata2 = sc.read_h5ad("../../artifacts/single_sample_method_benchmarking/nanostring_cosmx_human_nsclc_subsample_20pct_sagenet.h5ad")

In [76]:
adata2

AnnData object with n_obs × n_vars = 18895 × 960
    obs: 'cell_type'
    uns: 'sagenet_model_training_duration_run1', 'sagenet_model_training_duration_run10', 'sagenet_model_training_duration_run2', 'sagenet_model_training_duration_run3', 'sagenet_model_training_duration_run4', 'sagenet_model_training_duration_run5', 'sagenet_model_training_duration_run6', 'sagenet_model_training_duration_run7', 'sagenet_model_training_duration_run8', 'sagenet_model_training_duration_run9'
    obsm: 'sagenet_latent_run1', 'sagenet_latent_run10', 'sagenet_latent_run2', 'sagenet_latent_run3', 'sagenet_latent_run4', 'sagenet_latent_run5', 'sagenet_latent_run6', 'sagenet_latent_run7', 'sagenet_latent_run8', 'sagenet_latent_run9', 'spatial'

In [73]:
adata2.obsm["spatial"] = adata.obsm["spatial"]

In [74]:
adata2.write("../../artifacts/single_sample_method_benchmarking/nanostring_cosmx_human_nsclc_subsample_20pct_sagenet.h5ad")

#### 2.2.2 Compute Benchmarking Metrics

In [ ]:
compute_combined_benchmarking_metrics_for_all_models(
    dataset="nanostring_cosmx_human_nsclc_subsample_20pct",
    metrics=["gcs",
             "mlami",
             "cas",
             "clisis",
             "cnmi",
             "casw",
             "clisi",
             "nasw"],
    included_models=["scvi",
                     "expimap",
                     "graphst",
                     "sagenet"])

Computing metrics for scVI...
AnnData object with n_obs × n_vars = 18895 × 960
    obs: 'cell_type'
    uns: 'scvi_model_training_duration_run1', 'scvi_model_training_duration_run10', 'scvi_model_training_duration_run2', 'scvi_model_training_duration_run3', 'scvi_model_training_duration_run4', 'scvi_model_training_duration_run5', 'scvi_model_training_duration_run6', 'scvi_model_training_duration_run7', 'scvi_model_training_duration_run8', 'scvi_model_training_duration_run9'
    obsm: 'scvi_latent_run1', 'scvi_latent_run10', 'scvi_latent_run2', 'scvi_latent_run3', 'scvi_latent_run4', 'scvi_latent_run5', 'scvi_latent_run6', 'scvi_latent_run7', 'scvi_latent_run8', 'scvi_latent_run9', 'spatial'
Computing spatial nearest neighbor graph with 15 neighbors for entire dataset...
Computing spatial nearest neighbor graph with 90 neighbors for entire dataset...
Computing latent nearest neighbor graph with 15 neighbors for entire dataset...
Computing latent nearest neighbor graph with 90 neighbors 

/home/sbirk/storage/nichecompass/nichecompass/benchmarking/clisis.py:196: RuntimeWarning: invalid value encountered in log2
  cell_log_rclisi_scores = np.log2(cell_rclisi_scores)


CNMI and CARI metrics computed. Elapsed time: 0 minutes 50 seconds.

Computing CASW Metric...
CASW metric computed. Elapsed time: 0 minutes 51 seconds.

Computing CLISI metric...
CLISI metric computed. Elapsed time: 0 minutes 51 seconds.

Computing NASW Metric...
Using precomputed latent nearest neighbor graph...
Computing latent Leiden clusterings...
Using precomputed latent Leiden clusters for resolution 0.1.
Using precomputed latent Leiden clusters for resolution 0.55.
Using precomputed latent Leiden clusters for resolution 1.0.
Computing NASW...
NASW metric computed. Elapsed time: 0 minutes 53 seconds.

AnnData object with n_obs × n_vars = 18895 × 960
    obs: 'cell_type', 'leiden_spatial_0.1', 'leiden_spatial_0.55', 'leiden_spatial_1.0', 'leiden_latent_0.1', 'leiden_latent_0.55', 'leiden_latent_1.0'
    uns: 'expimap_model_training_duration_run1', 'expimap_model_training_duration_run10', 'expimap_model_training_duration_run2', 'expimap_model_training_duration_run3', 'expimap_model

/home/sbirk/storage/nichecompass/nichecompass/benchmarking/clisis.py:196: RuntimeWarning: invalid value encountered in log2
  cell_log_rclisi_scores = np.log2(cell_rclisi_scores)


CNMI and CARI metrics computed. Elapsed time: 0 minutes 39 seconds.

Computing CASW Metric...
CASW metric computed. Elapsed time: 0 minutes 39 seconds.

Computing CLISI metric...
CLISI metric computed. Elapsed time: 0 minutes 39 seconds.

Computing NASW Metric...
Using precomputed latent nearest neighbor graph...
Computing latent Leiden clusterings...
Using precomputed latent Leiden clusters for resolution 0.1.
Using precomputed latent Leiden clusters for resolution 0.55.
Using precomputed latent Leiden clusters for resolution 1.0.
Computing NASW...
NASW metric computed. Elapsed time: 0 minutes 42 seconds.

AnnData object with n_obs × n_vars = 18895 × 960
    obs: 'cell_type', 'leiden_spatial_0.1', 'leiden_spatial_0.55', 'leiden_spatial_1.0', 'leiden_latent_0.1', 'leiden_latent_0.55', 'leiden_latent_1.0'
    uns: 'expimap_model_training_duration_run1', 'expimap_model_training_duration_run10', 'expimap_model_training_duration_run2', 'expimap_model_training_duration_run3', 'expimap_model

...

CNMI and CARI metrics computed. Elapsed time: 0 minutes 39 seconds.

Computing CASW Metric...
CASW metric computed. Elapsed time: 0 minutes 40 seconds.

Computing CLISI metric...
CLISI metric computed. Elapsed time: 0 minutes 40 seconds.

Computing NASW Metric...
Using precomputed latent nearest neighbor graph...
Computing latent Leiden clusterings...
Using precomputed latent Leiden clusters for resolution 0.1.
Using precomputed latent Leiden clusters for resolution 0.55.
Using precomputed latent Leiden clusters for resolution 1.0.
Computing NASW...
NASW metric computed. Elapsed time: 0 minutes 42 seconds.

AnnData object with n_obs × n_vars = 18895 × 960
    obs: 'cell_type', 'leiden_spatial_0.1', 'leiden_spatial_0.55', 'leiden_spatial_1.0', 'leiden_latent_0.1', 'leiden_latent_0.55', 'leiden_latent_1.0'
    uns: 'expimap_model_training_duration_run1', 'expimap_model_training_duration_run10', 'expimap_model_training_duration_run2', 'expimap_model_training_duration_run3', 'expimap_model

/home/sbirk/storage/nichecompass/nichecompass/benchmarking/clisis.py:196: RuntimeWarning: invalid value encountered in log2
  cell_log_rclisi_scores = np.log2(cell_rclisi_scores)


CNMI and CARI metrics computed. Elapsed time: 0 minutes 38 seconds.

Computing CASW Metric...
CASW metric computed. Elapsed time: 0 minutes 39 seconds.

Computing CLISI metric...
CLISI metric computed. Elapsed time: 0 minutes 39 seconds.

Computing NASW Metric...
Using precomputed latent nearest neighbor graph...
Computing latent Leiden clusterings...
Using precomputed latent Leiden clusters for resolution 0.1.
Using precomputed latent Leiden clusters for resolution 0.55.
Using precomputed latent Leiden clusters for resolution 1.0.
Computing NASW...
NASW metric computed. Elapsed time: 0 minutes 41 seconds.

AnnData object with n_obs × n_vars = 18895 × 960
    obs: 'cell_type', 'leiden_spatial_0.1', 'leiden_spatial_0.55', 'leiden_spatial_1.0', 'leiden_latent_0.1', 'leiden_latent_0.55', 'leiden_latent_1.0'
    uns: 'expimap_model_training_duration_run1', 'expimap_model_training_duration_run10', 'expimap_model_training_duration_run2', 'expimap_model_training_duration_run3', 'expimap_model

/home/sbirk/storage/nichecompass/nichecompass/benchmarking/clisis.py:196: RuntimeWarning: invalid value encountered in log2
  cell_log_rclisi_scores = np.log2(cell_rclisi_scores)


CNMI and CARI metrics computed. Elapsed time: 0 minutes 39 seconds.

Computing CASW Metric...
CASW metric computed. Elapsed time: 0 minutes 39 seconds.

Computing CLISI metric...
CLISI metric computed. Elapsed time: 0 minutes 39 seconds.

Computing NASW Metric...
Using precomputed latent nearest neighbor graph...
Computing latent Leiden clusterings...
Using precomputed latent Leiden clusters for resolution 0.1.
Using precomputed latent Leiden clusters for resolution 0.55.
Using precomputed latent Leiden clusters for resolution 1.0.
Computing NASW...
NASW metric computed. Elapsed time: 0 minutes 41 seconds.


Computing metrics for SageNet...
AnnData object with n_obs × n_vars = 18895 × 960
    obs: 'cell_type'
    uns: 'sagenet_model_training_duration_run1', 'sagenet_model_training_duration_run10', 'sagenet_model_training_duration_run2', 'sagenet_model_training_duration_run3', 'sagenet_model_training_duration_run4', 'sagenet_model_training_duration_run5', 'sagenet_model_training_duratio

In [78]:
import pickle
with open("../../artifacts/single_sample_method_benchmarking/benchmarking_dict_list.pickle", 'rb') as file:
    loaded_dict = pickle.load(file)

In [79]:
loaded_dict

[{'gcs': 0.759037103955944,
  'mlami': 0.13767904287743543,
  'cas': 0.7065524966142978,
  'clisis': 0.9034444391727448,
  'cnmi': 0.09971506610187823,
  'casw': 0.45452479273080826,
  'clisi': 0.7321885333341711,
  'nasw': 0.6593832721312841,
  'model_name': 'sagenet',
  'run': 1},
 {'gcs': 0.7590114757333839,
  'mlami': 0.1252585331989316,
  'cas': 0.7127399685719442,
  'clisis': 0.9029718264937401,
  'cnmi': 0.08797399606863242,
  'casw': 0.444561131298542,
  'clisi': 0.7318247907301959,
  'nasw': 0.6632546087106069,
  'model_name': 'sagenet',
  'run': 2},
 {'gcs': 0.7586709766370209,
  'mlami': 0.11497138303395976,
  'cas': 0.6925338832362962,
  'clisis': 0.9003952816128731,
  'cnmi': 0.0753725428647786,
  'casw': 0.4383682385087013,
  'clisi': 0.7282575158511891,
  'nasw': 0.7294813692569733,
  'model_name': 'sagenet',
  'run': 3},
 {'gcs': 0.7585579413609067,
  'mlami': 0.1085961213253914,
  'cas': 0.6943345288118247,
  'clisis': 0.9001866057515144,
  'cnmi': 0.08456241899688136,

In [ ]:
compute_combined_benchmarking_metrics_for_all_models(
    dataset="vizgen_merfish_mouse_liver")

In [ ]:
compute_combined_benchmarking_metrics_for_all_models(
    dataset="vizgen_merfish_mouse_liver_sample")

In [ ]:
compute_combined_benchmarking_metrics_for_all_models(
    dataset="starmap_plus_mouse_cns")

In [ ]:
compute_combined_benchmarking_metrics_for_all_models(
    dataset="starmap_plus_mouse_cns_sample")

In [ ]:
compute_combined_benchmarking_metrics_for_all_models(
    dataset="nanostring_cosmx_human_nsclc")

In [ ]:
compute_combined_benchmarking_metrics_for_all_models(
    dataset="nanostring_cosmx_human_nsclc_subsample_20pct",
    )

#### 3.2.3 Visualize Results

In [ ]:
df = pd.DataFrame()
datasets = ["seqfish_mouse_organogenesis_embryo2",
            "starmap_plus_mouse_cns_sample",
            "nanostring_cosmx_human_nsclc_sample"]
timestamps = ["07032023_193322",
              "07032023_071219",
              "07032023_080409"]

for dataset, timestamp in zip(datasets, timestamps):
    dataset_artifact_folder_path = f"../../artifacts/{dataset}/method_benchmarking/comparison/{timestamp}"
    
    # Read complete benchmarking data from disk
    with open(f"{dataset_artifact_folder_path}/benchmarking_dict_list.pickle", "rb") as f:
        benchmarking_dict_list = pickle.load(f)
        dataset_df = pd.DataFrame(benchmarking_dict_list)
        dataset_df["dataset"] = dataset
        df = pd.concat([df, dataset_df])
df.head()

columns = ["gcs",
           "mlami",
           "cas",
           "clisis",
           #"gerr2",
           #"cca"
           ]

rows = ["nichecompass",
        "deeplinc",
        "graphst",
        "sagenet",
        "pca",
        "scvi",
        "expimap"]

unrolled_df = pd.melt(df, 
   id_vars = ["model_name", "dataset"],
   value_vars = columns,
   var_name = "score_type", 
   value_name = "score")

# Compute metric means over all runs
mean_df = unrolled_df.groupby(["model_name", "dataset", "score_type"]).mean()
mean_df.reset_index(inplace=True)

# Remove pca for plot
mean_df = mean_df[mean_df["model_name"] != "pca"]

# Reformat for plot
mean_df.replace({"nichecompass": "NicheCompass",
                 "deeplinc": "DeepLinc",
                 "expimap": "expiMap",
                 "graphst": "GraphST",
                 "sagenet": "SageNet",
                 "scvi": "scVI",
                 "nanostring_cosmx_human_nsclc_sample": "nanostring_cosmx_sample",
                 "starmap_plus_mouse_cns_sample": "starmap_plus_sample",
                 "seqfish_mouse_organogenesis_embryo2": "seqfish"}, inplace=True)

# Create spatial indicator column
def is_spatially_aware_model(row):
    if row["model_name"] in ["NicheCompass", "DeepLinc", "GraphST", "SageNet"]:
        return True
    return False
mean_df["spatially_aware"] = mean_df.apply(lambda row: is_spatially_aware_model(row), axis=1)

mean_df_seqfish = mean_df[mean_df["dataset"] == "seqfish"]

In [ ]:
plot_benchmarking_results(mean_df_seqfish,
                          save_dir=figure_folder_path,
                          save_name="benchmarking_results_seqfish.svg")

In [ ]:
plot_benchmarking_results(mean_df,
                          save_dir=figure_folder_path,
                          save_name="benchmarking_results.svg")

In [ ]:
# Compute metric means over all runs
mean_df = unrolled_df.groupby(["model_name", "dataset", "score_type"]).mean()

print(mean_df)
mean_df = mean_df.reindex(rows)

metric_ranks = []
for metric in mean_df.columns:
    mean_df[f"{metric}_rank"] = mean_df[metric].rank(ascending=False).astype(int)

mean_df["model_name"] = mean_df.index

unrolled_mean_df = pd.melt(mean_df, 
   id_vars = ["model_name", "dataset"],
   value_vars = columns,
   var_name = "score_type", 
   value_name = "score")

In [ ]:
plt.figure(figsize=(30, 15))
ax = sns.barplot(data=unrolled_mean_df,
                 x="score_type",
                 y="score",
                 hue="model_name",
                 palette="Accent")

plt.xticks(fontsize=30)
plt.yticks(fontsize=30)

# Iterate over the bars and annotate each one with the rank
for i, p in enumerate(ax.patches):
    model_idx, metric_idx = divmod(i, len(columns))
    rank = mean_df.loc[rows[model_idx], columns[metric_idx] + "_rank"]
    ax.annotate("%.0f" % rank,
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha="center",
                va="center",
                fontsize=20,
                color="gray",
                weight="bold",
                xytext=(0, 8),
                textcoords="offset points")

plt.xlabel("Metric",
           fontsize=30)
plt.ylabel("Metric Value (higher better)",
           fontsize=30)
plt.legend(loc=2,
           bbox_to_anchor=(1, 1),
           prop={"size": 28})
plt.title("Model Benchmarking Evaluation Metrics",
          fontsize=30,
          pad=20)
plt.savefig(f"{figure_folder_path}/eval_metrics.svg",
            dpi=300,
            bbox_inches="tight",
            format="svg")
plt.show()

In [ ]:
mean_df["dataset_score_type"] = mean_df["score_type"] + mean_df["dataset"]

In [ ]:
mean_df[mean_df["model_name"] == "nichecompass"]

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(30, 15))

plt.xlabel("Metric",
           fontsize=30)
plt.ylabel("Metric Value (higher better)",
           fontsize=30)
plt.legend(loc=2,
           bbox_to_anchor=(1, 1),
           prop={"size": 28})
plt.title("Model Benchmarking Evaluation Metrics",
          fontsize=30,
          pad=20)

sns.scatterplot(data=mean_df[mean_df["dataset"] == "starmap_plus_mouse_cns_sample"],
                ax=axs[0],
                 x="score_type",
                 y="score",
                 hue="model_name",
                 palette="Accent",
                 s=1000,
                 style="dataset")

sns.scatterplot(data=mean_df[mean_df["dataset"] == "nanostring_cosmx_human_nsclc_sample"],
                ax=axs[1],
                 x="score_type",
                 y="score",
                 hue="model_name",
                 palette="Accent",
                 s=1000,
                 style="dataset")

plt.xticks(fontsize=30)
plt.yticks(fontsize=30)


plt.savefig(f"{figure_folder_path}/eval_metrics.svg",
            dpi=300,
            bbox_inches="tight",
            format="svg")
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.scatterplot(data=mean_df,
                 x="dataset_score_type",
                 y="score",
                 hue="model_name",
                 palette="Accent",
                 s=500,
                 style="dataset")

plt.xticks(fontsize=30)
plt.yticks(fontsize=30)

# Iterate over the bars and annotate each one with the rank
for i, p in enumerate(ax.patches):
    model_idx, metric_idx = divmod(i, len(columns))
    rank = mean_df.loc[rows[model_idx], columns[metric_idx] + "_rank"]
    ax.annotate("%.0f" % rank,
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha="center",
                va="center",
                fontsize=20,
                color="gray",
                weight="bold",
                xytext=(0, 8),
                textcoords="offset points")

plt.xlabel("Metric",
           fontsize=30)
plt.ylabel("Metric Value (higher better)",
           fontsize=30)
plt.legend(loc=2,
           bbox_to_anchor=(1, 1),
           prop={"size": 28})
plt.title("Model Benchmarking Evaluation Metrics",
          fontsize=30,
          pad=20)
plt.savefig(f"{figure_folder_path}/eval_metrics.svg",
            dpi=300,
            bbox_inches="tight",
            format="svg")
plt.show()

##### 3.2.8.1 Metrics Ranking Plot

In [ ]:
mean_df_min_best = mean_df[["gcd", "cad", "arclisi", "germse"]] # lower values are better
mean_df_max_best = mean_df[["mlnmi", "cca", "ari", "clisi", "nmi", "asw", "ilasw", ]] # higher values are better
rank_df_min = mean_df_min_best.rank(method="max", ascending=True)
rank_df_max = mean_df_max_best.rank(method="max", ascending=False)
rank_df = pd.concat([rank_df_min, rank_df_max], axis=1)
rank_df = rank_df[columns]

In [ ]:
heatmap = sns.heatmap(rank_df, annot=True, cmap="YlGnBu")
fig = heatmap.get_figure()
plt.title("Method Benchmarking Metrics Ranking", fontsize=20, pad=25)
plt.xticks(rotation=45)
fig.savefig(f"{figure_folder_path}/metrics_ranking_{current_timestamp}.png",
            bbox_inches="tight")
plt.show()

### 2.1 Latent Space Comparison

#### 2.1.2 Visualize Results

##### 2.1.2.1 seqFISH Mouse Organogenesis

In [ ]:
compute_latent_space_comparison(
    dataset="seqfish_mouse_organogenesis_embryo2",
    cell_type_colors=color_utils.seqfish_mouse_organogenesis_cell_type_colors,
    cell_type_groups=None,
    n_neighbors=12,
    run_number=5,
    spot_size=0.03)

In [ ]:
compute_latent_space_comparison(dataset="seqfish_mouse_organogenesis_embryo2",
                                cell_type_colors=seqfish_mouse_organogenesis_embryo2_cell_type_colors,
                                cell_type_groups="Mixed mesenchymal mesoderm", # "Gut tube",
                                n_neighbors=12,
                                run_number=5,
                                spot_size=0.03)

##### 2.1.2.2 MERFISH Mouse Liver

In [ ]:
# DeepLinc did not work on this dataset
compute_latent_space_comparison(dataset="vizgen_merfish_mouse_liver",
                                cell_type_colors=vizgen_merfish_mouse_liver_cell_type_colors,
                                n_neighbors=12,
                                run_number=5,
                                included_models=["NicheCompass",
                                                 "expiMap",
                                                 "scVI"],
                                spot_size=20.0)

In [ ]:
# DeepLinc did not work on this dataset
compute_latent_space_comparison(dataset="vizgen_merfish_mouse_liver_sample",
                                cell_type_colors=vizgen_merfish_mouse_liver_cell_type_colors,
                                n_neighbors=12,
                                run_number=5,
                                included_models=["NicheCompass",
                                                 "GraphST",
                                                 "SageNet",
                                                 "expiMap",
                                                 "scVI"],
                                spot_size=20.0)

##### 2.1.2.3 STARmap PLUS Mouse CNS

In [ ]:
compute_latent_space_comparison(dataset="starmap_plus_mouse_cns",
                                cell_type_colors=starmap_pluse_mouse_cns_cell_type_colors,
                                n_neighbors=12,
                                run_number=5,
                                included_models=["NicheCompass",
                                                 "expiMap",
                                                 "scVI"],
                                spot_size=0.2)

In [ ]:
compute_latent_space_comparison(dataset="starmap_plus_mouse_cns_sample",
                                cell_type_colors=starmap_pluse_mouse_cns_cell_type_colors,
                                n_neighbors=12,
                                run_number=5,
                                spot_size=0.2)

##### 2.1.2.4 NanoString CosMx Human NSCLC

In [ ]:
compute_latent_space_comparison(dataset="nanostring_cosmx_human_nsclc",
                                cell_type_colors=nanostring_cosmx_human_nsclc_cell_type_colors,
                                n_neighbors=12,
                                run_number=5,
                                included_models=["NicheCompass",
                                                 "expiMap",
                                                 "scVI"],
                                spot_size=100.0)

In [ ]:
compute_latent_space_comparison(dataset="nanostring_cosmx_human_nsclc_sample",
                                cell_type_colors=nanostring_cosmx_human_nsclc_cell_type_colors,
                                n_neighbors=12,
                                run_number=5,
                                spot_size=100.0)